# Floodplain Mapping via GUI Automation (Last Resort)

This notebook demonstrates floodplain mapping using **HEC-RAS GUI automation**, which is the most fragile and least recommended method.

## When to Use This Method

✅ **Best for:**
- Debugging why maps aren't generating
- Visual verification in RASMapper GUI
- One-off manual operations

❌ **Not suitable for:**
- Production workflows (fragile)
- Batch processing (slow)
- Automated pipelines (GUI dependency)
- Cloud/headless environments (requires GUI)

## Comparison with Other Methods

| Method | Speed | Reliability | Matches HEC-RAS to 0.01' | Cloud-Compatible | GUI Required | Recommendation |
|--------|-------|-------------|--------------------------|------------------|--------------|----------------|
| 15_b: RasProcess CLI | ⭐⭐⭐ Fastest (8-10 sec) | ⭐⭐⭐ Excellent | ✅ Yes (100% - native) | ❌ No | ❌ No | **Recommended** (Windows) |
| 15_c: Python-GIS | ⭐⭐ Moderate (15-20 sec) | ⭐⭐⭐ Excellent | ✅ Yes (horizontal)* | ✅ Yes | ❌ No | **Recommended** (Cloud/2D) |
| **15_a: GUI Automation** | ⭐ Slow (60+ sec) | ⭐ Fragile | ✅ Yes (100% - native) | ❌ No | ✅ Yes | **Last Resort** |

\* Python-GIS matches HEC-RAS to 0.01' for **2D horizontal interpolation only** (validation tests in notebook 15_c). Limitations: 2D mesh only (no 1D support yet), horizontal interpolation only.

**Recommendation**: Use 15_b (RasProcess) or 15_c (Python-GIS) instead. Only use this method for debugging or visual verification.

### Reference Documentation

**HEC-RAS Mapping Resources**:
- [HEC-RAS User's Manual, Chapter 15: Using RAS Mapper](https://www.hec.usace.army.mil/software/hec-ras/documentation.aspx)
- [HEC-RAS 2D Modeling User's Manual](https://www.hec.usace.army.mil/software/hec-ras/documentation.aspx) (mapping chapter)

**Floodplain Mapping Standards**:
- [FEMA Guidelines and Standards for Flood Risk Analysis and Mapping](https://www.fema.gov/flood-maps/guidance-partners/guidelines-standards)
- [FEMA Guidance for Flood Risk Analysis and Mapping](https://www.fema.gov/flood-maps/tools-resources/engineering-library)

**Professional Context**: This notebook demonstrates automation of floodplain mapping workflows. All automated outputs should be reviewed by a licensed professional engineer before use in regulatory applications.

### Workflow Overview
This notebook uses **GUI automation** via `win32com` to drive RAS Mapper. This approach:
- ✓ Leverages HEC-RAS's native mapping engine
- ✓ Produces identical results to manual GUI operation
- ✓ Enables batch processing of multiple scenarios
- ✓ Creates reviewable HEC-RAS project files

## Prerequisites

- HEC-RAS 6.x installed (GUI automation requires HEC-RAS executable)
- Windows operating system
- pywin32 package (for Win32 COM automation)
- Computed HEC-RAS plan with results
- **User must be present** to monitor GUI operations

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import RasCmdr, RasExamples, RasMap, init_ras_project, ras

# Additional imports for visualization
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander
✓ Loaded: c:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\ras_commander\__init__.py


## Parameters

Configure these values to customize the notebook for your project.

In [2]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# HDF Analysis Settings
PLAN = "06"                       # Plan number (for HDF file path)
TIME_INDEX = -1                   # Time step index (-1 = last)
PROFILE = "Max"                   # Profile name for steady analysis

print(f"Configuration set: Project={PROJECT_NAME}, Plan={PLAN}, Profile={PROFILE}")

Configuration set: Project=BaldEagleCrkMulti2D, Plan=06, Profile=Max


## Step 1: Initialize Project

In [3]:
# Extract example project
project_path = RasExamples.extract_project(PROJECT_NAME, suffix="600")

# Initialize project
init_ras_project(project_path, RAS_VERSION)

print(f"Project: {ras.project_name}")
print(f"Folder: {ras.project_folder}")

2025-12-29 00:18:22 - ras_commander.RasExamples - INFO - Found zip file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\Example_Projects_6_6.zip
2025-12-29 00:18:22 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-12-29 00:18:22 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-12-29 00:18:22 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-12-29 00:18:22 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D' as 'BaldEagleCrkMulti2D_600'
2025-12-29 00:18:24 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600
2025-12-29 00:18:24 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagle

Project: BaldEagleDamBrk
Folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600


## Step 2: Pre-requisite - Check .rasmap Compatibility

**IMPORTANT**: For HEC-RAS 5.0.7 projects running in 6.x, the `.rasmap` file must be upgraded to 6.x format.

The `postprocess_stored_maps()` function now automatically detects this and provides instructions if manual intervention is needed.

## Step 3: Compute Plan (if needed)

Make sure plan 06 has been computed.

In [4]:
# Check if plan 06 has HDF results
hdf_path = ras.project_folder / f"{ras.project_name}.p06.hdf"

if not hdf_path.exists():
    print("Computing plan 06...")
    RasCmdr.compute_plan(PLAN, num_cores=2)
else:
    print("Plan 06 already computed")

2025-12-29 00:18:24 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600
2025-12-29 00:18:24 - ras_commander.RasUtils - INFO - Using provided plan file path: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600\BaldEagleDamBrk.p06
2025-12-29 00:18:24 - ras_commander.RasUtils - INFO - Successfully updated file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600\BaldEagleDamBrk.p06
2025-12-29 00:18:24 - ras_commander.RasCmdr - INFO - Set number of cores to 2 for plan: 06
2025-12-29 00:18:24 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-12-29 00:18:24 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\Users\billk_clb\anaconda3\envs\rascmdr_p

Computing plan 06...


2025-12-29 00:27:19 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 06
2025-12-29 00:27:19 - ras_commander.RasCmdr - INFO - Total run time for plan 06: 535.36 seconds


In [ ]:
# Display results summary from results_df
ras.results_df[['plan_number', 'plan_title', 'completed', 'has_errors', 'has_warnings', 'runtime_complete_process_hours']]

## Step 4: Generate Maps via GUI Automation

### How This Works

The `postprocess_stored_maps()` function performs GUI automation:

1. **Checks .rasmap compatibility** (upgrades 5.0.7→6.x if needed)
2. **Modifies .rasmap XML** to add stored map layer definitions
3. **Updates plan flags** to run floodplain mapping only (skips full simulation)
4. **Opens HEC-RAS GUI** via Win32 COM automation
5. **Navigates menus** to click "Run > Unsteady Flow Analysis"
6. **Waits for completion** (user must close HEC-RAS when done)
7. **Restores original files** (plan, .rasmap) while keeping generated maps

### Available Map Types

- `Depth` - Water depth (maximum)
- `WSEL` - Water surface elevation (maximum)
- `Velocity` - Flow velocity (maximum)
- All other variables supported by RASMapper

### User Interaction Required

⚠️ **IMPORTANT**: You will need to:
1. Monitor the HEC-RAS window that opens
2. Wait for floodplain mapping to complete (watch status bar)
3. Close HEC-RAS when computation finishes

The function will wait for you to close HEC-RAS before continuing.

In [5]:
# Get available terrains
rasmap_path = ras.project_folder / f"{ras.project_name}.rasmap"
terrains = RasMap.get_terrain_names(rasmap_path)
print(f"Available terrains: {terrains}")

# Use first terrain if available
target_terrain = terrains[0] if terrains else None
print(f"Using terrain: {target_terrain}")

# Generate stored maps via GUI automation
print("\n" + "="*60)
print("GUI AUTOMATION STARTING")
print("HEC-RAS will open automatically.")
print("Please monitor the window and close HEC-RAS when done.")
print("="*60 + "\n")

success = RasMap.postprocess_stored_maps(
    plan_number=PLAN,
    specify_terrain=target_terrain,
    layers=['Depth', 'WSEL']  # Generate Depth and WSE maps
)

if success:
    print("\n✅ Successfully generated stored maps via GUI automation")
else:
    print("\n❌ Failed to generate stored maps")

2025-12-29 00:27:19 - ras_commander.rasmap - INFO - Extracted terrain names: ['Terrain50']
2025-12-29 00:27:19 - ras_commander.rasmap - INFO - Checking .rasmap compatibility...
2025-12-29 00:27:19 - ras_commander.rasmap - INFO - .rasmap file is already compatible (version 2.0.0)
2025-12-29 00:27:19 - ras_commander.rasmap - INFO - .rasmap compatibility check passed: Already compatible (version 2.0.0)
2025-12-29 00:27:19 - ras_commander.rasmap - INFO - Backing up plan file C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600\BaldEagleDamBrk.p06 to C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600\BaldEagleDamBrk.p06.storedmap.bak
2025-12-29 00:27:19 - ras_commander.rasmap - INFO - Updating plan run flags for floodplain mapping for plan 06...
2025-12-29 00:27:19 - ras_commander.RasPlan - INFO - Successfully updated run flags in plan file: C:\Users\billk_clb\

Available terrains: ['Terrain50']
Using terrain: Terrain50

GUI AUTOMATION STARTING
HEC-RAS will open automatically.
Please monitor the window and close HEC-RAS when done.



2025-12-29 00:27:20 - ras_commander.RasGuiAutomation - INFO - Found 'already running' dialog - clicking Yes to continue
2025-12-29 00:27:20 - ras_commander.RasGuiAutomation - INFO - Clicked button: &Yes
2025-12-29 00:27:20 - ras_commander.RasGuiAutomation - INFO - Clicked 'Yes' button on already running dialog
2025-12-29 00:27:21 - ras_commander.RasGuiAutomation - INFO - Waiting for HEC-RAS main window...
2025-12-29 00:27:23 - ras_commander.RasGuiAutomation - INFO - Found HEC-RAS main window: HEC-RAS 6.6
2025-12-29 00:27:24 - ras_commander.RasGuiAutomation - INFO - Clicking 'Run > Unsteady Flow Analysis' menu...
2025-12-29 00:27:24 - ras_commander.RasGuiAutomation - INFO - Clicked menu item ID: 47
2025-12-29 00:27:26 - ras_commander.RasGuiAutomation - INFO - Looking for Unsteady Flow Analysis dialog...
2025-12-29 00:27:26 - ras_commander.RasGuiAutomation - INFO - Found Unsteady Flow Analysis dialog
2025-12-29 00:27:26 - ras_commander.RasGuiAutomation - INFO - Looking for Compute button


✅ Successfully generated stored maps via GUI automation


## Step 5: Visualize Results

In [6]:
# Find the output directory
plan_info = ras.plan_df[ras.plan_df['plan_number'] == '06'].iloc[0]
short_id = plan_info.get('Short Identifier', 'Plan_06')
output_folder = ras.project_folder / short_id

# Find depth map
depth_map_path = output_folder / "Depth (Max).vrt"

if depth_map_path.exists():
    print(f"Found depth map at: {depth_map_path}")
    
    with rasterio.open(depth_map_path) as src:
        fig, ax = plt.subplots(figsize=(10, 8))
        show(src, ax=ax, cmap='Blues', title='Maximum Depth (GUI Automation)')
        plt.tight_layout()
        plt.show()
        
        print(f"CRS: {src.crs}")
        print(f"Resolution: {src.res}")
        print(f"Bounds: {src.bounds}")
else:
    print(f"Depth map not found at: {depth_map_path}")

Depth map not found at: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_600\Grid Precip Infiltration\Depth (Max).vrt


## Technical Explanation: Win32 COM Automation

### Overview

The GUI automation method uses **Win32 COM API** to control the HEC-RAS application programmatically.

### How It Works

#### 1. Open HEC-RAS Application

```python
import subprocess
ras_exe = "C:/Program Files/HEC/HEC-RAS/6.6/RAS.exe"
prj_path = "C:/path/to/project.prj"

# Open HEC-RAS with project
process = subprocess.Popen([ras_exe, prj_path])
pid = process.pid
```

#### 2. Find HEC-RAS Window by Process ID

```python
import win32gui
import win32process

def find_window_by_pid(pid):
    """Enumerate all windows to find HEC-RAS window"""
    def callback(hwnd, windows):
        if win32gui.IsWindowVisible(hwnd):
            _, window_pid = win32process.GetWindowThreadProcessId(hwnd)
            if window_pid == pid:
                title = win32gui.GetWindowText(hwnd)
                if "HEC-RAS" in title:
                    windows.append(hwnd)
        return True
    
    windows = []
    win32gui.EnumWindows(callback, windows)
    return windows[0] if windows else None
```

#### 3. Enumerate Menu System

```python
def enumerate_menus(hwnd):
    """Find 'GIS Tools' > 'RAS Mapper' menu item"""
    menu_bar = win32gui.GetMenu(hwnd)
    menu_count = win32gui.GetMenuItemCount(menu_bar)
    
    for i in range(menu_count):
        # Get menu text
        submenu = win32gui.GetSubMenu(menu_bar, i)
        item_count = win32gui.GetMenuItemCount(submenu)
        
        for j in range(item_count):
            # Check if this is "Run > Unsteady Flow Analysis"
            menu_id = win32gui.GetMenuItemID(submenu, j)
            # Return menu ID if found
```

#### 4. Click Menu via WM_COMMAND

```python
import win32api
import win32con

# Send command to HEC-RAS to click menu
WM_COMMAND = 0x0111
win32api.PostMessage(hwnd, WM_COMMAND, menu_id, 0)
```

#### 5. Wait for Computation to Complete

```python
# Wait for user to close HEC-RAS
while win32gui.IsWindow(hwnd):
    time.sleep(1)

print("HEC-RAS closed, computation complete")
```

### Why This Is Fragile

**Menu IDs Can Change**:
- Different HEC-RAS versions use different menu IDs
- Updates can break automation

**Window Titles Can Vary**:
- "HEC-RAS 6.6" vs "HEC-RAS 6.5" vs "HEC-RAS"
- Dialog titles may differ by version

**Timing Issues**:
- Need to wait for windows to load
- Race conditions if timing is wrong
- No reliable way to detect completion

**Focus Issues**:
- Other windows can steal focus
- User interaction can break automation
- Window minimization causes failures

**Dialog Variations**:
- Confirmation dialogs may or may not appear
- Error dialogs can halt automation
- No standardized window hierarchy

### Performance

**Typical timing** (BaldEagleCrkMulti2D, 2 variables):
- GUI Automation: 60+ seconds
- RasProcess CLI: 8-10 seconds (6-7x faster)
- Python-GIS: 15-20 seconds (3-4x faster)

**Overhead breakdown**:
- HEC-RAS startup: 5-10 seconds
- Menu navigation: 2-3 seconds
- Dialog handling: 2-3 seconds
- Actual mapping: 40-50 seconds
- User close wait: Variable

### When GUI Automation Is Useful

Despite the fragility, GUI automation has value:

✅ **Debugging**: See what HEC-RAS is doing in real-time

✅ **Visual Verification**: Confirm maps look correct in RASMapper

✅ **One-off Operations**: When speed doesn't matter

✅ **Learning**: Understand HEC-RAS workflow

❌ **Production**: Use RasProcess CLI (15_b) instead

❌ **Batch Processing**: Use RasProcess CLI (15_b) or Python-GIS (15_c)

❌ **Cloud/Headless**: Use Python-GIS (15_c) only

## Troubleshooting

### Issue: HEC-RAS Doesn't Open

**Solution**: Verify HEC-RAS installation:
```python
# Check if HEC-RAS executable exists
print(f"HEC-RAS path: {ras.ras_exe_path}")
print(f"Exists: {ras.ras_exe_path.exists()}")
```

### Issue: Menu Not Found

**Symptom**: "Could not find menu item"

**Cause**: Menu IDs changed in different HEC-RAS version

**Solution**: Use keyboard shortcuts as fallback (function tries this automatically)

### Issue: Function Hangs Waiting for HEC-RAS

**Symptom**: "Waiting for HEC-RAS to close..."

**Solution**: Manually close HEC-RAS application (computation may have failed)

### Issue: No Maps Generated

**Symptom**: Output folder empty or VRT files missing

**Diagnosis**:
1. Check if floodplain mapping actually ran
2. Look for errors in HEC-RAS compute log
3. Verify .rasmap was upgraded (5.0.7→6.x)
4. Ensure terrain exists in .rasmap

**Solution**: Try RasProcess CLI method (15_b) instead - more reliable

### Issue: .rasmap Upgrade Failed

**Symptom**: "Manual intervention required"

**Solution**: Follow on-screen instructions:
1. Open project in HEC-RAS
2. Click "GIS Tools" > "RAS Mapper"
3. Allow .rasmap update when prompted
4. Close RASMapper and HEC-RAS
5. Re-run this notebook

## Decision Matrix: When to Use Each Method

### Use 15_a (GUI Automation) when:
- ✅ Debugging why maps aren't generating
- ✅ Visual verification in RASMapper needed
- ✅ One-off manual operation
- ✅ Learning HEC-RAS workflow
- ⚠️ **Last resort when other methods fail**

### Use 15_b (RasProcess CLI) when:
- ✅ Running on Windows (most common)
- ✅ Need fastest performance
- ✅ Need all variables (Froude, Shear, D*V, etc.)
- ✅ Batch processing multiple plans
- ✅ Production workflows
- ✅ **RECOMMENDED DEFAULT**

### Use 15_c (Python-GIS) when:
- ✅ Running in cloud/Docker
- ✅ Linux/Mac environment
- ✅ Only need WSE, Depth, Velocity
- ✅ 2D mesh projects only
- ✅ Reproducible research workflows
- ✅ **RECOMMENDED for cloud/headless**

## Summary

This notebook demonstrated:

1. ✅ GUI automation approach using `RasMap.postprocess_stored_maps()`
2. ✅ Automatic .rasmap compatibility checking
3. ✅ Win32 COM technical explanation
4. ✅ Performance comparison (60+ seconds vs 8-10 seconds RasProcess)
5. ✅ Fragility issues and when to avoid
6. ✅ Decision matrix showing this is last resort

**Key Takeaway**: GUI automation is the slowest and most fragile method. Use RasProcess CLI (15_b) for Windows or Python-GIS (15_c) for cloud. Only use GUI automation for debugging or visual verification.

**Next Steps**:
- See notebook 15_b for recommended Windows method (RasProcess CLI)
- See notebook 15_c for cloud-compatible method (Python-GIS)
- See notebook 11 for detailed HDF data extraction